## Overview
- Don't return a value from an expression until it's needed
- Python allocates memory for objects, and it won't be freed as long as the object exists in the program
    - This isn't a problem for small objects, but larger objects require more memory, which can cause performance issues

- Lazy evaluation delays when a program evaluates expressions, which can improve the performance of a program by spreading the time-consuming process across a longer time period
    - It also prevents values that won't be used from being generated

- Errors raised by a lazily-evaluated expression are deferred to later in the program, which can make debugging more difficult

## Examples
`enumerate()` creates an iterator behind the scenes

In [5]:
import random

names = ['Sarah', 'Matt', 'Jim', 'Denise', 'Kate']
random.shuffle(names)

numbered_names = enumerate(names, start=1)
numbered_names

In [6]:
print(next(numbered_names))
print(next(numbered_names))

(1, 'Jim')
(2, 'Kate')


And so does the `zip()` function

In [10]:
names = ['Sarah', 'Matt', 'Jim', 'Denise', 'Kate']
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
random.shuffle(names)

day_name_pairs = zip(weekdays, names)
print(next(day_name_pairs))
print(next(day_name_pairs))

# We can modify the third element in "names" and the third element in "day_name_pairs" will reflect that change since it's evaluated lazily!
names[2] = 'The Coffee Robot'
print(next(day_name_pairs))

# This can be nice, but we need to be cautious as the data is not fixed at the time of creation

('Monday', 'Matt')
('Tuesday', 'Sarah')
('Wednesday', 'The Coffee Robot')


## Itertools

`chain()` - combine 2 iterators into a new iterator, which is then evaluated lazily

In [11]:
import itertools

first_team = ['Sarah', 'Matt', 'Jim']
second_team = ['Denise', 'Kate']

for name in itertools.chain(first_team, second_team):
    print(name)

Sarah
Matt
Jim
Denise
Kate


`islice()`

In [12]:
numbers = [1, 2, 3, 4, 5]
standard_slice = numbers[1:4]
iterator_slice = itertools.islice(numbers, 1, 4)

iterator_slice

In [13]:
numbers[2] = 999

print('standard_slice:', list(standard_slice))
print('iterator_slice:', list(iterator_slice))

standard_slice: [2, 3, 4]
iterator_slice: [2, 999, 4]


## Generator Expressions and Generator Functions
Generator objects are iterators whose values are generated as needed./

In [14]:
coin_toss = (
    'H' if random.random() > 0.5 else 'T'
    for _ in range(10)
)

coin_toss

<generator object <genexpr> at 0x10768de50>

In [15]:
print(next(coin_toss))
print(next(coin_toss))

T
H


In [16]:
[res for res in coin_toss]  # only 8 tosses left until the generator is exhausted

['H', 'H', 'H', 'H', 'T', 'H', 'T', 'T']